In [4]:
## make dir
import os
import numpy as np

os.makedirs('midis/test', exist_ok=True)
converter_script = 'musescore'

In [5]:
import subprocess

os.makedirs('data/midis_wavs', exist_ok=True)

for file in os.listdir('data/midis'):
    if 'mid' in file:
        midi = 'midis/' + file
        wav = "data/midis_wavs/" + file.replace('mid', 'flac')
        subprocess.Popen(f'{converter_script} "{midi}" -o "{wav}"', shell=True).wait()

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: eglfs, linuxfb, minimal, minimalegl, offscreen, vnc, xcb.

Aborted (core dumped)
qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: eglfs, linuxfb, minimal, minimalegl, offscreen, vnc, xcb.

Aborted (core dumped)


KeyboardInterrupt: 

In [ ]:
from transformers import AutoFeatureExtractor, AutoModel
import torch

model_id = 'ntu-spml/distilhubert'
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

sampling_rate = feature_extractor.sampling_rate
model = AutoModel.from_pretrained(model_id)

In [ ]:
from datasets import load_dataset, Audio

path = 'C:\\Users\\konin\\PHd\\distilhubert encoder (13000 midis)\\midis'
dataset = load_dataset('audiofolder', data_dir=path)

# resample audio files to desired sample rate
dataset = dataset.cast_column('audio', Audio(sampling_rate=sampling_rate))

In [ ]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        return_attention_mask=True,
    )
    return inputs

dataset_encoded = dataset.map(
    preprocess_function,
    remove_columns=['audio'],
    batched=True,
    batch_size=100,
    num_proc=1,
)

dataset_encoded

In [ ]:
import librosa

result_dict = {}

for i in range(len(os.listdir("midis\\test"))):
    if i%1000==0:
        print(i)
    t = torch.FloatTensor(dataset_encoded['test'][i]['input_values'] )
    tt = t.view(1, t.shape[0])
    
    wav = dataset['test'][i]['audio']['path']

    h = model(tt).last_hidden_state
    h_mean = np.mean(h.squeeze().detach().numpy(), axis = 0)
    
    result_dict[wav.split('\\')[-1]] = h_mean

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(result_dict, orient='index', dtype=np.float32).reset_index()
df.to_csv('13000 midis results.csv', index=False)

In [1]:
import pandas as pd
import numpy as np

df_new = pd.read_csv('13000 midis results.csv')
df_new.iloc[:, 1:] = df_new.iloc[:, 1:].astype('float32')
np_array = df_new.to_numpy()

In [7]:
names = np_array[:, 0]
features = np_array[:, 1:]

(12960, 768)

In [ ]:
''' 
# Some plots

import plotly.graph_objects as go
import numpy as np

# Assuming `h` is your torch tensor
h_numpy = h.squeeze().detach().numpy()

# Create a time array for y-axis
duration = t.shape[0] / sampling_rate
time_seconds = np.linspace(0, duration, h_numpy.shape[0])

# Create traces for each feature
traces = []
for i in range(h_numpy.shape[1]):
    trace = go.Scatter(x=time_seconds, y=h_numpy[:, i], mode='lines', name=f'Feature {i}')
    traces.append(trace)

# Create the plot
fig = go.Figure(data=traces)

# Update layout
fig.update_layout(title='Plot of Features Over Time',
                  xaxis_title='Time (seconds)',
                  yaxis_title='Feature Value')

# Show the plot
fig.show()

'''

In [ ]:
'''
import numpy as np
import matplotlib.pyplot as plt

# Assuming `data` is your dataset with shape (num_samples, 768)

# Perform FFT
fft_result = np.fft.fft(h_numpy, axis=0)
frequencies = np.fft.fftfreq(h_numpy.shape[0], 1 / sampling_rate)

# Plot the frequency spectrum
plt.figure(figsize=(10, 6))
plt.plot(frequencies[:h_numpy.shape[0]//2], np.abs(fft_result[:h_numpy.shape[0]//2]))
plt.title('Frequency Spectrum')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()
'''